<a href="https://colab.research.google.com/github/wczubal1/BloombergTask/blob/main/BloombergTaskWitoldCzubala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from statistics import NormalDist

In [2]:
#!pip install scikit-learn

In [3]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
#Uploading corporate PD datset
data_root = "https://raw.githubusercontent.com/wczubal1/BloombergTask/main/"
df_credit = pd.read_csv(data_root + "df_credit.csv")

In [5]:
df_credit

,Instrument,CompanyName,BusinessSector,CompanyMarketCap,CreditCombinedPD,ReturnOnAssets,TotalDebttoCommonEquity,PriceMomentumT12MVolatility
0,TXG.OQ,10X Genomics Inc,Healthcare Services & Equipment,5.554622e+09,0.083735,-16.21,NaN,14.111801
1,FCNCA.OQ,First Citizens BancShares Inc (Delaware),Banking & Investment Services,2.005774e+10,0.251288,1.01,0.92905,24.651832
2,ME.OQ,23andMe Holding Co.,Healthcare Services & Equipment,3.392539e+08,0.235883,-29.76,NaN,14.719923
3,TSVT.OQ,2Seventy Bio Inc,Pharmaceuticals & Medical Research,1.893360e+08,0.414782,NaN,NaN,34.163690
4,DDD.N,3D Systems Corp,Technology Equipment,6.925227e+08,0.459605,-1.96,0.60469,21.900110
...,...,...,...,...,...,...,...,...
2324,ZI.OQ,Zoominfo Technologies Inc,Software & IT Services,6.127379e+09,0.216172,5.20,0.54393,18.089863
2325,ZS.OQ,Zscaler Inc,Software & IT Services,3.423091e+10,0.049608,8.65,1.56412,16.097909
2326,ZUMZ.OQ,Zumiez Inc,Retailers,3.589960e+08,0.233710,2.61,0.00000,13.146578
2327,ZUO.N,Zuora Inc,Software & IT Services,1.209195e+09,0.081718,NaN,2.16553,16.002749


In [6]:
#Count of rows with missing values
print (df_credit.isna().any(axis=1).sum())

802


In [7]:
#Removing missing values
df_credit=df_credit.dropna()

In [36]:
df_credit.loc[:,'randNumCol'] = np.random.uniform(0, 1, df_credit.shape[0])

<ipython-input-36-d4b71def6516>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit.loc[:,'randNumCol'] = np.random.uniform(0, 1, df_credit.shape[0])


In [25]:
df_credit['CreditCombinedPD'].describe()
#df_credit['randNumCol'].describe()

count    1527.000000
mean        0.283032
std         0.493445
min         0.006694
25%         0.074972
50%         0.140495
75%         0.269909
max         5.972882
Name: CreditCombinedPD, dtype: float64

In [39]:
#simulating default
#scaling parameter
k=3
df_credit.loc[:,'Default'] = df_credit['randNumCol']<(k*df_credit['CreditCombinedPD']/100)

<ipython-input-39-ee7159d617f5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit.loc[:,'Default'] = df_credit['randNumCol']<(k*df_credit['CreditCombinedPD']/100)


In [40]:
sum(df_credit['Default'])
#df_credit.loc[df_credit['Default']==True]
#df_credit.loc[df_credit['Default']==True,'BusinessSector'].describe()

16

In [41]:
#Sector variables
df_credit.groupby('BusinessSector').size()
#Creating dummies for sector
df_credit=pd.concat([df_credit, pd.get_dummies(df_credit['BusinessSector'],drop_first=True)],axis=1)

In [42]:
#Estimation dataset (dropping several columns)
df_estimation= df_credit[df_credit.columns.difference(['Instrument', 'CompanyName', 'BusinessSector', 'CompanyMarketCap', 'CreditCombinedPD', 'randNumCol'])]

In [43]:
df_estimation.columns

Index(['Automobiles & Auto Parts', 'Banking & Investment Services',
       'Chemicals', 'Consumer Goods Conglomerates',
       'Cyclical Consumer Products', 'Cyclical Consumer Services', 'Default',
       'Energy - Fossil Fuels',
       'Financial Technology (Fintech) & Infrastructure', 'Food & Beverages',
       'Food & Drug Retailing', 'Healthcare Services & Equipment',
       'Industrial & Commercial Services', 'Industrial Goods', 'Insurance',
       'Mineral Resources', 'Personal & Household Products & Services',
       'Pharmaceuticals & Medical Research', 'PriceMomentumT12MVolatility',
       'Real Estate', 'Renewable Energy', 'Retailers', 'ReturnOnAssets',
       'Software & IT Services', 'Technology Equipment',
       'Telecommunications Services', 'TotalDebttoCommonEquity',
       'Transportation', 'Utilities'],
      dtype='object')

In [44]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_estimation.drop(['Default'],axis=1), df_estimation['Default'], test_size=0.25)

# Create the logistic regression model
#Adding regularization penalty and inverse of regularization strength
model = LogisticRegression(penalty='l2', C=0.01)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

In [45]:
# Evaluate the model
accuracy = model.score(X_test, y_test)
print('Accuracy:', accuracy)
accuracy = model.score(X_train, y_train)
print('Accuracy:', accuracy)

Accuracy: 0.9895287958115183
Accuracy: 0.988646288209607


In [46]:
model.coef_
#confusion_matrix(y_test, model.predict(X_test))
#confusion_matrix(y_train, model.predict(X_train))

array([[ 0.00697647, -0.00785292, -0.00219656, -0.00031789, -0.00391166,
         0.01273398, -0.00466423, -0.00149994,  0.00745489, -0.00095806,
         0.00952008, -0.00468393,  0.00298763, -0.00295489, -0.00122822,
        -0.00130586,  0.00394979,  0.05462328,  0.00218811, -0.00166794,
         0.00598002, -0.01265748, -0.01324417, -0.00699287, -0.00164144,
         0.02294498, -0.00282164,  0.00733938]])

In [ ]:
print(classification_report(y_train, model.predict(X_train)))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00      1139
        True       1.00      0.17      0.29         6

    accuracy                           1.00      1145
   macro avg       1.00      0.58      0.64      1145
weighted avg       1.00      1.00      0.99      1145



In [ ]:
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}